In [8]:
from pynq import Overlay
import pynq.lib as lib
ol= Overlay("spi.bit")

In [9]:
ol?

In [10]:
spi =ol.axi_quad_spi_0

In [12]:
data_ch0 = spi.read(0)
print(data_ch0)

0


In [13]:
# SPI initialization function
def init(spi):
    spi.write(0x40, 0x0a)       # Configure the SPI
    spi.write(0x28, 0x04)
    spi.write(0x1c, 0x00)
    spi.write(0x70, 0xFFFFFFFF)
    
    ctrlreg = spi.read(0x60)    # Read control register
    ctrlreg = ctrlreg | 0xe6    # Modify control register
    spi.write(0x60, ctrlreg)
    
    ctrlreg = spi.read(0x60)    # Ensure the control register is correctly set
    ctrlreg = ctrlreg & ~(0x15)
    spi.write(0x60, ctrlreg)


In [14]:
# SPI transfer function
def transfer(packet, spi):
    for data in packet:
        spi.write(0x68, data)   # Write data to Tx FIFO
        spi.write(0x70, 0xFFFFFFFE)   # Start transfer
        
        ctrlreg = spi.read(0x60)
        ctrlreg = ctrlreg & ~(0x100)  # Clear the transfer complete flag
        spi.write(0x60, ctrlreg)
        
        statReg = spi.read(0x64)      # Check status register
        while (statReg & 0x04) == 0:  # Wait for transfer to complete
            statReg = spi.read(0x64)
        
        ctrlreg = spi.read(0x60)
        ctrlreg = ctrlreg | 0x100     # Set the transfer complete flag
        spi.write(0x60, ctrlreg)
    
    spi.write(0x70, 0xFFFFFFFF)       # Reset the SPI interface
    
    recvData = list()
    RxFifoStatus = spi.read(0x64) & 0x01  # Check if data is in Rx FIFO
    while RxFifoStatus == 0:
        temp = spi.read(0x6c)         # Read data from Rx FIFO
        recvData.append(temp)
        RxFifoStatus = spi.read(0x64) & 0x01
    return recvData

        
    

In [15]:
init(spi)
ctrlreg =spi.read(0x60)
print(ctrlreg)

386


In [ ]:
# Main program to collect 100 data points
init(spi)
packet = [0x00]
data = transfer(packet, spi)
print(data)
    

In [ ]:
# Main program to collect 100 data points
init(spi)
data_points = []

for i in range(100):
    packet = [0x00]  # Adjust the packet content as per your SPI device
    data = transfer(packet, spi)
    data_points.append(data)

print(data_points)  # Print the collected data points
